In [61]:
# from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
import re



In [2]:
path_to_file = 'data/poe.txt'


In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))
print(text[:250])


Length of text: 357133 characters
THE RAVEN.


  Once upon a midnight dreary, while I pondered, weak and weary,
  Over many a quaint and curious volume of forgotten lore--
  While I nodded, nearly napping, suddenly there came a tapping,
  As of some one gently rapping--rapping 


In [4]:
clean_text = re.sub(r"[^a-zA-Z0-9]+", ' ', text)


In [5]:
len(clean_text)

305182

In [6]:
vocab = sorted(set(clean_text))
print ('{} unique characters'.format(len(vocab)))

63 unique characters


In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in clean_text])


In [ ]:
print('{')
for char,_ in zip(char2idx, range(30)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

In [36]:
seq_length = 100
examples_per_epoch = len(clean_text)//seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

T
H
E
 
R


In [37]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'THE RAVEN Once upon a midnight dreary while I pondered weak and weary Over many a quaint and curious '
'volume of forgotten lore While I nodded nearly napping suddenly there came a tapping As of some one g'
'ently rapping rapping at my chamber door Tis some visitor I muttered tapping at my chamber door Only '
'this and nothing more Ah distinctly I remember it was in the bleak December And each separate dying e'
'mber wrought its ghost upon the floor Eagerly I wished the morrow vainly I had sought to borrow From '


In [38]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [39]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'THE RAVEN Once upon a midnight dreary while I pondered weak and weary Over many a quaint and curious'
Target data: 'HE RAVEN Once upon a midnight dreary while I pondered weak and weary Over many a quaint and curious '


In [40]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 30 ('T')
  expected output: 18 ('H')
Step    1
  input: 18 ('H')
  expected output: 15 ('E')
Step    2
  input: 15 ('E')
  expected output: 0 (' ')
Step    3
  input: 0 (' ')
  expected output: 28 ('R')
Step    4
  input: 28 ('R')
  expected output: 11 ('A')


In [41]:
BATCH_SIZE = 32
steps_per_epoch = examples_per_epoch//BATCH_SIZE
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((32, 100), (32, 100)), types: (tf.int64, tf.int64)>

In [42]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [43]:
import functools
rnn = functools.partial(
        tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [44]:
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(len(vocab), embedding_dim, 
                              batch_input_shape=[BATCH_SIZE, None]),
        rnn(rnn_units,
            return_sequences=True, 
            recurrent_initializer='glorot_uniform',
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
      ])

In [45]:
for input_example_batch, target_example_batch in dataset.take(1): 
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(32, 100, 63) # (batch_size, sequence_length, vocab_size)


In [46]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (32, None, 256)           16128     
_________________________________________________________________
gru_3 (GRU)                  (32, None, 1024)          3935232   
_________________________________________________________________
dense_3 (Dense)              (32, None, 63)            64575     
Total params: 4,015,935
Trainable params: 4,015,935
Non-trainable params: 0
_________________________________________________________________


In [47]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [48]:
sampled_indices

array([13, 38, 37, 36, 51, 25,  4, 37,  4,  0, 39, 32, 61, 40, 30,  2, 36,
       22, 19, 26, 18, 39, 13, 59, 51, 54,  5, 47,  2, 29, 21, 32, 61, 44,
        9,  1, 37, 43, 46, 34, 60, 56, 29, 21, 18, 48, 12, 60, 10, 54, 42,
        5,  7, 35, 39, 14, 51, 17, 32, 27, 40, 45, 51, 60, 54, 37,  7, 20,
        1, 44, 37, 17, 15, 20, 12, 12, 56,  6, 32, 37, 51,  2, 33, 12, 54,
        5, 19, 41,  7, 27,  6, 54, 35, 26, 52, 58, 21, 50, 35, 49])

In [53]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)

In [50]:
print(example_batch_loss)

tf.Tensor(
[[4.1435127 4.1346817 4.159623  ... 4.136654  4.121233  4.14752  ]
 [4.135777  4.1652446 4.144858  ... 4.1495605 4.165277  4.13259  ]
 [4.1405115 4.1301775 4.1546454 ... 4.1384187 4.15343   4.1503186]
 ...
 [4.148098  4.1356344 4.140433  ... 4.1471033 4.12692   4.1462245]
 [4.1435127 4.1504617 4.144043  ... 4.149508  4.123887  4.1422486]
 [4.1492524 4.1322384 4.1692677 ... 4.1469216 4.1464295 4.1453834]], shape=(32, 100), dtype=float32)


In [51]:
loss = tf.keras.losses.sparse_categorical_crossentropy(
    target_example_batch, example_batch_predictions, from_logits=True)
print(loss)

tf.Tensor(
[[4.1435127 4.1346817 4.159623  ... 4.136654  4.121233  4.14752  ]
 [4.135777  4.1652446 4.144858  ... 4.1495605 4.165277  4.13259  ]
 [4.1405115 4.1301775 4.1546454 ... 4.1384187 4.15343   4.1503186]
 ...
 [4.148098  4.1356344 4.140433  ... 4.1471033 4.12692   4.1462245]
 [4.1435127 4.1504617 4.144043  ... 4.149508  4.123887  4.1422486]
 [4.1492524 4.1322384 4.1692677 ... 4.1469216 4.1464295 4.1453834]], shape=(32, 100), dtype=float32)


In [54]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [55]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [56]:
EPOCHS=3
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/3
95/95 [==============================] - 370s 4s/step - loss: 2.9208
Epoch 2/3
95/95 [==============================] - 437s 5s/step - loss: 2.1993
Epoch 3/3
95/95 [==============================] - 393s 4s/step - loss: 2.0041


In [57]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            16128     
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_4 (Dense)              (1, None, 63)             64575     
Total params: 4,015,935
Trainable params: 4,015,935
Non-trainable params: 0
_________________________________________________________________


In [60]:
model.save('my_model_02')

In [34]:
model = tf.keras.models.load_model("my_model_01", compile=False)

In [59]:
def generate_text(model, start_string):
    num_generate = 200
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 0.5
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))
print(generate_text(model, start_string="The"))

Ther and the project in the loly with prears of the sure and the buther And in the best now the arf the the earts of the sour I me The sor cander and the rest and som the sine the into the come in the pr
